# Testing Readers and SQL Alchemy Creation

In our eegdata storage code, we have two main functions:
1. parsing data from excel files, tables, online, text files, etc. into memory
2. writing data into the sql tables from memory
3. reading/updating data from the sql tables



In [39]:
import pandas as pd
import numpy as np
import os
from time import time

In [40]:
datafile = '/Users/adam2392/Dropbox/EZTrack-Prospective-Study/datasheet_fordb.xlsx'
data = pd.read_excel(datafile)

In [41]:
display(data.head())

,Patient ID,Age at surgery,Onset Age,Sampling Freq,Gender,Hand Dominant,Center,Number of Contacts,Number Contacts Used,Size of Clinical EZ,...,Resected contacts,Location of Surgery,Seizure Type,Imaging Clinical Notes,Localization from EEG,Post-op Progress Info,Date of First recurrence,Date of Treatment,Date of last follow-up,Time Since First Seizure
0,pt1,30.0,13.0,1000,F,R,NIH,98.0,86.0,10,...,"ATT1-8, AST1-4, PST1-4, AD1-4, PD1-4, PLT5-6, ...",R ATL,"Complex partial, at times with secondary gener...",large area of encephalomalacia in R parietal r...,likely dual pathology - R parietal and R mesia...,NaN,sensory auras only postop,2016-04-25,2017-04-26,NaN
1,pt2,28.0,19.0,1000,F,R,NIH,86.0,64.0,8,...,"G1-4, 9-12, 18-20, 26, 27, TT1-6, AST1-4, MST1-4",L ATL,focal; complex partial,L mesial temporal sclerosis,L temporal,seizure free,na,2016-03-14,2017-03-16,NaN
2,pt3,45.0,32.0,1000,M,R,NIH,135.0,99.0,12,...,"FG1,2,9,10,17,18,25, SFP1-8, MFP1-6, IFP1-4, O...",R frontal topectomy,focal; hypermotor,normal,R frontal,seizure free,na,2014-11-03,2017-03-02,NaN
3,pt6,33.0,25.0,1000,M,R,NIH,99.0,81.0,12,...,"LALT1-6, LAST1-4, LA 1-4, LPST 4, LAH 1-4, LPH1,2",L ATL,"complex partial, at times with secondary gener...",nonspecific,bitemporal,worthwhile improvement,7 months postop,2015-02-27,2016-12-09,NaN
4,pt7,39.0,18.0,1000,F,R,NIH,111.0,101.0,16,...,"G28-30, 36-39, 41,44-46, LFP 1,2, LSF 3,4",R frontal topectomy,focal; hypermotor,normal,R frontal,2017-04-26 00:00:00,~12 months postop,2012-11-16,2017-04-26,NaN


In [42]:
t = time()

colmapping = {
    'patient id': 'patid',
    'gender': 'gender',
    'age at surgery': 'age',
    'engel score': 'engel_score',
    'outcome': 'outcome',
    'hand dominant': 'hand_dominant',
    'seizure type': 'seizure_type',
    'location of surgery': 'surgery_location',
    'localization from eeg': 'eeg_localization',
    'date of treatment': 'surgery_date',
    'onset electrodes': 'onset_chans',
    'time since first seizure': 'epilepsy_duration'
}

for col in data:
    colname = col.lower()
    if colname in colmapping.keys():
        print(colname)

patient id
age at surgery
gender
hand dominant
outcome
engel score
onset electrodes
location of surgery
seizure type
localization from eeg
date of treatment
time since first seizure


In [43]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import sessionmaker
from models.models import db_connect, create_tables
from models.models import PatData, Chanxyz, Regxyz, EegRecord

#Create the database
# engine = create_engine('sqlite:///csv_test.db')
# Base.metadata.create_all(engine)

#Create the session
engine = db_connect()
create_tables(engine)
Session = sessionmaker(bind=engine)
s = Session()

try:
    records = dict()

#     for index, row in data.iterrows():
#         record = PatData(**{
#             ''
#         })
    for col in data:
        if col.lower() in colmapping.keys():
            print(col)
            # get the colname 
            colname = colmapping[col.lower()]
            records[colname.lower()] = data[col]

    db_record = PatData(**records)
    s.add(db_record)
except:
    s.rollback()
finally:
    s.close()
print(records.keys())

2018-04-02 11:02:41,624 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-04-02 11:02:41,625 INFO sqlalchemy.engine.base.Engine ()
2018-04-02 11:02:41,627 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-04-02 11:02:41,628 INFO sqlalchemy.engine.base.Engine ()
2018-04-02 11:02:41,629 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("patdata")
2018-04-02 11:02:41,630 INFO sqlalchemy.engine.base.Engine ()
2018-04-02 11:02:41,632 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("chanxyz")
2018-04-02 11:02:41,632 INFO sqlalchemy.engine.base.Engine ()
2018-04-02 11:02:41,634 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("regxyz")
2018-04-02 11:02:41,634 INFO sqlalchemy.engine.base.Engine ()
2018-04-02 11:02:41,636 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("eegrecord")
2018-04-02 11:02:41,637 INFO sqlalchemy.engine.base.Engine ()
Patient ID
Age at surgery
Gender
Hand

In [44]:
data.columns = map(str.lower, data.columns)

for col in data:
    print(col)
    try:
        data[col] = data[col].str.lower()
    except AttributeError as e:
        print(col, " has an attribute error when casting to strings.")

patient id
age at surgery
age at surgery  has an attribute error when casting to strings.
onset age 
onset age   has an attribute error when casting to strings.
sampling freq
sampling freq  has an attribute error when casting to strings.
gender
hand dominant
center
number of contacts
number of contacts  has an attribute error when casting to strings.
number contacts used
number contacts used  has an attribute error when casting to strings.
size of clinical ez
size of clinical ez  has an attribute error when casting to strings.
outcome
engel score
onset electrodes
resected contacts 
location of surgery
seizure type
imaging clinical notes
localization from eeg
post-op progress info
date of first recurrence
date of treatment
date of treatment  has an attribute error when casting to strings.
date of last follow-up
date of last follow-up  has an attribute error when casting to strings.
time since first seizure
time since first seizure  has an attribute error when casting to strings.


# Modify and Update the DB